In [22]:
from settings import *
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [23]:
#Settings
%matplotlib inline
pd.set_option('display.max_columns', 97)

def read_features():
    
    raw_data = pd.read_csv("2016-SFO-Customer-Survey-Data.csv")
    
    data = raw_data.drop(DROP_COLS, axis = 1)
    
    return data

def prep_missing(data):
    
    # Survey codes blank/multiple responses as zeros. Replace all 0s and "0"s with NaN. 
    data = data.replace(0, np.nan)
    data = data.replace("0", np.nan)
    
    # Calculate threshold based on percentage of columns
    #TODO - Missing Data: remove hardcoding of threshold level
    threshold = len(data.index) * .15
    
    # Find columns with missing data above a certain threshold
    high_missing = data.columns[data.isnull().sum() > threshold].tolist()
    
    # Keep only columns with low missing percentage from data
    imputables = data.drop(high_missing, axis=1)
    
    # Rename column names to human-readable text
    #TODO: Save long list of column names as dict to import from settings file
    imputables_renamed = imputables.rename(columns=COLUMN_NAMES)
    return imputables_renamed

def encode_categories(data):
    
    label_encoder = LabelEncoder()
    encoded_categories = data.apply(label_encoder.fit_transform)
    return encoded_categories
 
def get_categoricals(categorical_data):
    
    categorical_columns = CAT_COLS
    
    categorical_features = categorical_data[categorical_columns]
         
    #Convert to categories dtype   
    for col in categorical_columns:
        categorical_features[col] = categorical_features[col].astype('category')
        
    #Remove erroneous responses
    categorical_features['Trav_age_group'] = categorical_features['Trav_age_group'].cat.remove_categories(['Under 18','Under 19', 'Under 20', 'Under 21', 'Under 22','Under 23', 'Under 24', 'Under 25', 'Under 26', 'Under 27', 'Under 28', 'Under 29', 'Under 30', 'Under 31', 'Under 32']) 
    
    categorical_data['Trav_age_group'] = categorical_data['Trav_age_group'].replace("Don't Know or Refused", "NaN")
    
    encoded_cats = pd.get_dummies(categorical_features)
    
    # TODO: Combine times flown and sfo (maybe yearsUsed?)
    
    return categorical_features, encoded_cats

def get_numerics(numeric_data, cols_to_remove):
    
    numeric_data['Trav_Arr_to_dep_time'] = numeric_data['Trav_Arr_to_dep_time'].replace("N", "NaN")
    numeric_features = numeric_data.drop(cols_to_remove, axis=1)
    return numeric_features

def get_xy_vars(data, col_name):
    
    y = data[col_name]
    x = data.drop(col_name, axis=1)
    return y, x

def imputer(data, strategy):
    
    imp = Imputer(missing_values='NaN', strategy=strategy, axis=0)
    x_clean = imp.fit_transform(data)
    x_all = pd.DataFrame(x_clean, columns = data.columns) #Put column names back on
    return x_all

def barplot(data):
    pass


if __name__ == "__main__":
    
    data=read_features()
    
    prepped = prep_missing(data)
    
    cat_cols, dummy_cats = get_categoricals(prepped)
    nums = get_numerics(prepped, cat_cols)
    
    y_all, x_nums = get_xy_vars(nums, 'Rate_SFO_overall')
    
    imputed_nums = imputer(x_nums, 'median')
    
    x_all = pd.concat([imputed_nums,dummy_cats], axis=1)
    
    #barplot(cat_cols)
    #barplot(imputed_nums)
    print x_all

/Users/KoryLyles/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/KoryLyles/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


      Trav_Arr_to_dep_time  Used_free_wifi  Rate_art  Rate_food  Rate_stores  \
0                    215.0             1.0       6.0        2.0          4.0   
1                    110.0             2.0       3.0        3.0          3.0   
2                    125.0             2.0       4.0        4.0          4.0   
3                     80.0             2.0       6.0        6.0          6.0   
4                    120.0             1.0       5.0        5.0          5.0   
5                    100.0             2.0       2.0        3.0          6.0   
6                    120.0             1.0       3.0        4.0          4.0   
7                    315.0             1.0       4.0        2.0          5.0   
8                    150.0             1.0       5.0        5.0          4.0   
9                    170.0             2.0       6.0        4.0          5.0   
10                    87.0             1.0       4.0        4.0          3.0   
11                   245.0             2